<a href="https://colab.research.google.com/github/RodrigoVitorinoS/otimizacao_frequencia_estudantil/blob/main/C%C3%93DIGO_DA_AUTOMATIZA%C3%87%C3%83O_DA_AN%C3%81LISE_ESTAT%C3%8DSTICA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#Importação das bibliotécas

import pandas as pd
import numpy as np
from scipy.stats import shapiro, bartlett, levene, boxcox, boxcox_normmax
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
import statsmodels.api as sm
from statsmodels.formula.api import ols
import plotly.express as px

#Ler dados das turmas

dados = pd.read_csv('https://raw.githubusercontent.com/RodrigoVitorinoS/otimizacao_frequencia_estudantil/main/Freq.csv')
dados = dados.drop('Unnamed: 0', axis = 1)
#Visualização dos dados da turma

def visualizacao_turma(dados,turma):
      df_turma = dados.copy()
      df_turma = df_turma[['DS',turma]]
      df_turma.dropna(inplace = True)
      df_turma = pd.crosstab(df_turma.index, df_turma.DS, df_turma[turma], aggfunc='sum')
      df_turma = df_turma.apply(lambda col: col.dropna().reset_index(drop=True))
      df_turma = df_turma.fillna(0).astype(int)
      df_turma=df_turma[['seg', 'ter', 'qua', 'qui', 'sex']]
      df_turma.rename(columns = {'seg':'Segunda',
                        'ter':'Terça',
                        'qua':'Quarta',
                        'qui':'Quinta',
                        'sex':'Sexta'}, inplace = True)
      return df_turma

#Tratar dados

def tratamento(dados, turma):
      df_turma = dados.copy()
      df_turma = df_turma[['DS',turma]]
      df_turma.rename(columns={turma:'freq', 'Data':'data', 'DS':'ds'}, inplace = True)
      df_turma.dropna(inplace = True)
      df_turma.replace({'seg':'Segunda',
                        'ter':'Terça',
                        'qua':'Quarta',
                        'qui':'Quinta',
                        'sex':'Sexta'}, inplace = True)
      return df_turma

#Análise Exploratória
##Gráfico de pizza
def pizza_graf(dados, turma):
      dados = tratamento(dados, turma)
      dados = dados.groupby('ds').mean().reset_index()
      # Criando o gráfico interativo
      fig = px.pie(dados, values = 'freq', names = 'ds')
      fig.update_traces(textposition='inside', textinfo='percent+label',
                        pull = [0.2 if i  == dados['freq'].min() else 0 for i in dados['freq']])

      # Atualizar o layout do título para formatar o texto
      fig.update_layout(
          title={
              'text': f'<b>Porcentagem de Frequências da turma {turma}</b> <br>Por Dias da Semana',

              'font': {
                  'family': 'Times New Roman',
                  'size': 24,
                  'color': 'Black',

              }
          },
      )

      fig.show()

##Gráfico boxplot

def boxplot(dados, turma):
      df_turma = tratamento(dados, turma)
      # Criando o gráfico interativo
      fig = px.box(df_turma, y = 'freq', x = 'ds', color="ds",
                  category_orders={'ds':['Segunda', "Terça", 'Quarta', "Quinta", "Sexta"]},  labels={'ds':''})

      # Atualizar o layout do título para formatar o texto
      fig.update_layout(

          title={
              'text': f'<b>BoxPlot de Frequências da turma {turma}</b><br>Por Dias da Semana',
              'font': {
                  'family': 'Times New Roman',
                  'size': 22,
                  'color': 'Black',

              }
          },
          margin=dict(t=100),

        xaxis={
              'title': {
                  'text': ''
              },
              'tickfont': {
                  'family': 'Times New Roman',
                  'size': 17,

              }
          },

          yaxis={
              'title': {
                  'text': ''
              },
              'tickfont': {
                  'family': 'Times New Roman',
                  'size': 17,


              }
          },

      )
      fig.show()


#Métodos estastísticos

def anova(df_analise):
      # Criar o modelo de ANOVA
      modelo_anova = ols('freq ~ C(ds)', data=df_analise).fit()

      # Obter os resíduos
      residuos_anova = modelo_anova.resid
      anova_resultados = sm.stats.anova_lm(modelo_anova, typ=1)
      _, p_shapiro = shapiro(residuos_anova)
      _, p_bartlett = bartlett(*[df_analise['freq'][df_analise['ds'] == dia] for dia in df_analise['ds'].unique()])

      if p_shapiro > 0.05 and p_bartlett > 0.05:
          return p_shapiro, p_bartlett, residuos_anova, anova_resultados, False, 1

      else:
        lamb = boxcox_normmax(df_analise['freq'])
        df_transformado = df_analise.copy()
        df_transformado['freq'] = df_transformado['freq']**lamb
        modelo_anova = ols('freq ~ C(ds)', data=df_transformado).fit()
        residuos_anova = modelo_anova.resid
        anova_resultados = sm.stats.anova_lm(modelo_anova, typ=1)
        _, p_shapiro = shapiro(residuos_anova)
        _, p_bartlett = bartlett(*[df_transformado['freq'][df_transformado['ds'] == dia] for dia in df_transformado['ds'].unique()])

        if p_shapiro > 0.05 and p_bartlett > 0.05:
            return p_shapiro, p_bartlett, residuos_anova, anova_resultados, True, 1

        else:
            return p_shapiro, p_bartlett, residuos_anova, anova_resultados, True, 0

def analize(df_analise, turma):
      p_shapiro, p_bartlett, residuos_anova, anova_resultados, foram_transformados, op = anova(df_analise)


      display(HTML(f"<h3 style='padding-left: 28%;'>{turma}</h3>"))


      display(anova_resultados)

      display(HTML(f"<h3>Teste de normalidade de resíduos Shapiro-Wil {'(Dados Transformados)' if foram_transformados else ''}</h3>"))
      print(f'\033[1mValor-p:\033[0m {p_shapiro}')
      if p_shapiro <= 0.05:
          print('\033[1mATENÇÃO:\033[0m a 5% de significância, os resíduos não podem ser considerados normais!')


      display(HTML(f"<h3>Teste de homogeneidade de variância {'(Dados Transformados)' if foram_transformados else ''}</h3>"))
      print(f'\033[1mValor-p:\033[0m {p_bartlett}')
      if p_bartlett <= 0.05:
          print('\033[1mATENÇÃO:\033[0m a 5% de significância, os resíduos não podem ser considerados homogêneos!')

      print('\n')
      tukey_results = pairwise_tukeyhsd(endog=df_analise['freq'], groups=df_analise['ds'])
      # Converter o resultado para um DataFrame
      tukey_df = pd.DataFrame(data=tukey_results._results_table.data[1:], columns=tukey_results._results_table.data[0])
      display(HTML(f"<h3 style='padding-left: 28%;'>Teste De Tukey</h3>"))
      # Exibir o DataFrame
      display(tukey_df)






      return op, anova_resultados

def resultados_completos():
      turmas_normais = []
      turmas_nao_normais = []
      p_valores = []
      exemplo = {}
      for turma in dados.drop(['DS', 'Data'], axis = 1).columns:
        df_analise = tratamento(dados, turma)
        op, tabela_anova = analize(df_analise, turma)
        p_valores.append(round(tabela_anova.loc['C(ds)', 'PR(>F)'], 6))
        if op == 1:
          turmas_normais.append(turma)
        else:
          turmas_nao_normais.append(turma)
        print('\n')
      df_p_valores = pd.DataFrame({'Hipótese Nula': ['Verdadeira' if p_valor > 0.05 else 'Falsa' for p_valor in p_valores],
                          'Turmas': dados.drop(['DS', 'Data'], axis = 1).columns})
      df_p_valores.to_excel('df_p_valores.xlsx')
      display(HTML("<h3>Turmas que passaram nos testes de normalidade e homogeneidade</h3>"))
      print(turmas_normais)

      display(HTML("<h3>Turmas que NÃO passaram nos testes normalidade e homogeneidade</h3>"))
      print(turmas_nao_normais)




In [ ]:
resultados_completos()

,df,sum_sq,mean_sq,F,PR(>F)
C(ds),4.0,1.424009e+10,3.560022e+09,5.498805,0.000377
Residual,145.0,9.387553e+10,6.474174e+08,NaN,NaN


Valor-p: 0.2746434807777405


Valor-p: 0.9609690630786583




,group1,group2,meandiff,p-adj,lower,upper,reject
0,Quarta,Quinta,-0.1478,0.9999,-3.4911,3.1956,False
1,Quarta,Segunda,-4.1133,0.0077,-7.4566,-0.7700,True
2,Quarta,Sexta,-3.1607,0.0629,-6.4262,0.1047,False
3,Quarta,Terça,-1.2545,0.8260,-4.5199,2.0110,False
4,Quinta,Segunda,-3.9655,0.0103,-7.2794,-0.6516,True
5,Quinta,Sexta,-3.0129,0.0809,-6.2482,0.2224,False
6,Quinta,Terça,-1.1067,0.8788,-4.3420,2.1286,False
7,Segunda,Sexta,0.9526,0.9262,-2.2827,4.1879,False
8,Segunda,Terça,2.8588,0.1104,-0.3764,6.0941,False
9,Sexta,Terça,1.9062,0.4562,-1.2485,5.0610,False


,df,sum_sq,mean_sq,F,PR(>F)
C(ds),4.0,1.077327e+08,2.693318e+07,3.90725,0.004792
Residual,148.0,1.020183e+09,6.893128e+06,NaN,NaN


...

Valor-p: 0.7976041851932034




,group1,group2,meandiff,p-adj,lower,upper,reject
0,Quarta,Quinta,-0.1676,0.9998,-2.8006,2.4654,False
1,Quarta,Segunda,-1.0634,0.7704,-3.5838,1.4571,False
2,Quarta,Sexta,-3.9421,0.0004,-6.5035,-1.3807,True
3,Quarta,Terça,0.4563,0.9887,-2.1512,3.0639,False
4,Quinta,Segunda,-0.8958,0.8711,-3.4667,1.6751,False
5,Quinta,Sexta,-3.7745,0.0010,-6.3856,-1.1635,True
6,Quinta,Terça,0.6239,0.9665,-2.0324,3.2803,False
7,Segunda,Sexta,-2.8788,0.0151,-5.3763,-0.3812,True
8,Segunda,Terça,1.5197,0.4678,-1.0252,4.0646,False
9,Sexta,Terça,4.3985,0.0001,1.8131,6.9839,True


,df,sum_sq,mean_sq,F,PR(>F)
C(ds),4.0,5359.319427,1339.829857,1.157652,0.33204
Residual,146.0,168975.853994,1157.368863,NaN,NaN


Valor-p: 0.1603771597146988


Valor-p: 0.569869884006007


